In [1]:
from gammapy.modeling import Parameter

In [2]:
class LinkedParameter(Parameter):
    def __init__(self, par1, par2):
        self.par1 = par1
        self.par2 = par2
        self.check_units()
        self._value2 = par2.value
        
        super().__init__(
            self.par1.name, 
            self._value2, 
            self.par1.unit
        )

    def check_units(self):
        pass
       
    def update(self):
        if self.par2.value != self._value2:
            self._value2 = self.par2.value
            self.value = self._value2
        
    @property
    def value(self):
        """Factor (float)."""
        self.update()
        return self._value2
        
    @value.setter
    def value(self, val):
        self.factor = float(val) / self._scale

In [3]:
def __set__(self, instance, value):
    if isinstance(value, Parameter) and self.unit.is_equivalent(value.unit):
        if self.name == value.name:
            instance.__dict__[self.name] = value
        else:
            instance.__dict__[self.name] = LinkedParameter(self, value)            
    else:
        par = instance.__dict__[self.name]
        raise TypeError(f"Cannot assign {value!r} to parameter {par!r}")
Parameter.__set__ = __set__

In [4]:
# THIS SHOULD FAIL (UNITS DO NOT MATCH)
from gammapy.modeling.models import BrokenPowerLawSpectralModel
model = BrokenPowerLawSpectralModel()
model.index1 = model.amplitude
model.index1

TypeError: Cannot assign Parameter(name='amplitude', value=1e-12, factor=1e-12, scale=1.0, unit=Unit("1 / (cm2 s TeV)"), min=nan, max=nan, frozen=False, id=0x7f542e320460) to parameter Parameter(name='index1', value=2.0, factor=2.0, scale=1.0, unit=Unit(dimensionless), min=nan, max=nan, frozen=False, id=0x7f542e320a00)

In [5]:
# THIS SHOULD NOT FAIL 
from gammapy.modeling.models import BrokenPowerLawSpectralModel, PowerLawSpectralModel
model1 = BrokenPowerLawSpectralModel(index1=2)
model2 = PowerLawSpectralModel(index=3)
model1.index1 = model2.index
model1.index1

LinkedParameter(name='index1', value=3.0, factor=3.0, scale=1.0, unit=Unit(dimensionless), min=nan, max=nan, frozen=False, id=0x7f543c5307f0)

Check that the parameters are correctly linked

In [6]:
model2.index.value = 18
model1.index1

LinkedParameter(name='index1', value=18.0, factor=18.0, scale=1.0, unit=Unit(dimensionless), min=nan, max=nan, frozen=False, id=0x7f543c5307f0)